In [341]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import sqlite3
import pickle
pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from functools import reduce 

In [342]:
conn = sqlite3.connect('database.db')

In [343]:
#Train

query_train = '''
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2012
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2013
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2014
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2015
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2016
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code","Falls within", COUNT() AS amount_of_crimes
    FROM street2017
    GROUP BY "Month", "LSOA code"
    UNION ALL
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2018
    GROUP BY "Month", "LSOA code"
    '''
dataframe2 = pd.read_sql_query(query_train, conn)

In [344]:
#Test

query_test = '''
    SELECT DISTINCT "Month", "LSOA name", "LSOA code", "Falls within", COUNT() AS amount_of_crimes
    FROM street2019
    GROUP BY "Month", "LSOA code"
    '''

dataframe3 = pd.read_sql_query(query_test, conn)

In [345]:
#Train
crime_per_district = dataframe2.copy()
crime_per_district["Falls within"] = crime_per_district["Falls within"].str.rsplit(' ',1).str[0]
crime_per_district = crime_per_district.rename(columns={'LSOA name': 'name', 'LSOA code': 'Geo code', 'Falls within':'Force Name'})



#Test
crime_per_district2 = dataframe3.copy()
crime_per_district2["Falls within"] = crime_per_district2["Falls within"].str.rsplit(' ',1).str[0]
crime_per_district2 = crime_per_district2.rename(columns={'LSOA name': 'name', 'LSOA code': 'Geo code', 'Falls within':'Force Name'})

# Police

In [350]:
df_police = pd.read_excel('Number of police officers per district per year.ods', engine='odf')
df_police = df_police.rename(columns={'As at 31 March...': 'year', 'Geo code': 'Geo code police'})

#Train
bb = df_police[df_police['year']>2011]
cc = bb[bb['year']<2019]
aa = cc[cc['Worker type']=='Police Officer']
df_train = aa.groupby(['year','Force Name', 'Geo code police'])[['Total (headcount)']].sum()

#Test
ee = df_police[df_police['year']>2018]
ff = ee[ee['year']<2020]
dd = ff[ff['Worker type']=='Police Officer']
df_test = dd.groupby(['year','Force Name', 'Geo code police'])[['Total (headcount)']].sum()

# Population

In [355]:
df_2012 = pd.read_excel('people2012.xlsx')
df_2013 = pd.read_excel('people2013.xlsx')
df_2014 = pd.read_excel('people2014.xlsx')
df_2015 = pd.read_excel('people2015.xlsx')
df_2016 = pd.read_excel('people2016.xlsx')
df_2017 = pd.read_excel('people2017.xlsx')
df_2018 = pd.read_excel('people2018.xlsx')
df_2019 = pd.read_excel('people2019.xlsx')

C:\Users\blaze\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [356]:
df_2012 = df_2012[['Area Codes', 'All Ages']]
df_2013 = df_2013[['Area Codes', 'All Ages']]
df_2014 = df_2014[['Area Codes', 'All Ages']]
df_2015 = df_2015[['Area Codes', 'All Ages']]
df_2016 = df_2016[['Area Codes', 'All Ages']]
df_2017 = df_2017[['Area Codes', 'All Ages']]
df_2018 = df_2018[['Area Codes', 'All Ages']]
df_2019 = df_2019[['LSOA Code', 'All Ages']]

In [357]:
df_2012 = df_2012.rename(columns={'All Ages':'Population 2012'})
df_2013 = df_2013.rename(columns={'All Ages':'Population 2013'})
df_2014 = df_2014.rename(columns={'All Ages':'Population 2014'})
df_2015 = df_2015.rename(columns={'All Ages':'Population 2015'})
df_2016 = df_2016.rename(columns={'All Ages':'Population 2016'})
df_2017 = df_2017.rename(columns={'All Ages':'Population 2017'})
df_2018 = df_2018.rename(columns={'All Ages':'Population 2018'})
df_2019 = df_2019.rename(columns={'LSOA Code': 'Area Codes', 'All Ages':'Population 2019'})

In [358]:
data_frames = [df_2012, df_2013,df_2014,df_2015,df_2016,df_2017,df_2018,df_2019]

In [359]:
df_pop_merged =  reduce(lambda  left,right: pd.merge(left,right,on=['Area Codes'], how='outer'), data_frames)

In [361]:
zz2 = df_pop_merged.copy()
zz2 = zz2.loc[zz2.index.repeat(8)]
zz2['year']= 2012
zz2['LSOA population']= 514261.0
zz2.reset_index(inplace=True)

In [362]:
for i in range (1, zz2.shape[0]):
    if zz2['index'][i] == zz2['index'][i-1]:
        zz2['year'][i] = zz2['year'][i-1]+1

C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/228228283.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zz2['year'][i] = zz2['year'][i-1]+1


In [363]:
for i in range (0, zz2.shape[0]):
    if zz2['year'][i]==2012:
        zz2['LSOA population'][i] = zz2['Population 2012'][i]
    if zz2['year'][i]==2013:
        zz2['LSOA population'][i] = zz2['Population 2013'][i]
    if zz2['year'][i]==2014:
        zz2['LSOA population'][i] = zz2['Population 2014'][i]
    if zz2['year'][i]==2015:
        zz2['LSOA population'][i] = zz2['Population 2015'][i]
    if zz2['year'][i]==2016:
        zz2['LSOA population'][i] = zz2['Population 2016'][i]
    if zz2['year'][i]==2017:
        zz2['LSOA population'][i] = zz2['Population 2017'][i]
    if zz2['year'][i]==2018:
        zz2['LSOA population'][i] = zz2['Population 2018'][i]
    if zz2['year'][i]==2019:
        zz2['LSOA population'][i] = zz2['Population 2019'][i] 

C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/3200078983.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zz2['LSOA population'][i] = zz2['Population 2012'][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/3200078983.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zz2['LSOA population'][i] = zz2['Population 2013'][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/3200078983.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

In [364]:
zz2.drop(['Population 2012', 'Population 2013','Population 2014','Population 2015','Population 2016','Population 2017', 'Population 2018', 'Population 2019'], axis=1, inplace=True)

In [365]:
zz2 = zz2.rename(columns={'Area Codes': 'LSOA code'})

In [368]:
df_population = pd.read_csv("Population of each district.csv")
df_population=df_population.rename(columns={'ladcode21':'Geo code', 'laname21':'name'})

df_districts = pd.read_csv("districts.csv")
df_districts = df_districts.rename(columns={'LAD19CD':'Geo code', 'LAD19NM':'name','PFA19CD': 'Geo code police', 'PFA19NM':'Force Name'})
df_districts.drop(['CSP19CD', 'CSP19NM', 'FID'], axis=1, inplace=True)

#Train
df_train2 = df_population.groupby(['name', 'Geo code'])[['population_2012', 'population_2013', 'population_2014', 'population_2015', 'population_2016', 'population_2017', 'population_2018', 'population_2019']].sum()
df_train2.reset_index(inplace=True)

abc_train = pd.merge(df_train2, df_districts, on='Geo code')
abc_train = abc_train.rename(columns={'name_x': 'name'})
abc_train.drop(['name_y'], axis=1, inplace=True)

#Test
df_test2 = df_population.groupby(['name', 'Geo code'])[['population_2019']].sum()
df_test2.reset_index(inplace=True)

abc_test = pd.merge(df_test2, df_districts, on='Geo code')
abc_test = abc_test.rename(columns={'name_x': 'name'})
abc_test.drop(['name_y'], axis=1, inplace=True)

# Merge population with the police

In [371]:
#Train
df_train3 = df_train.unstack(level=0)
df_train3.reset_index(inplace=True)

#Test
df_test3 = df_test.unstack(level=0)
df_test3.reset_index(inplace=True)

In [372]:
#Train
df_merged_train = pd.merge(abc_train, df_train3, on='Force Name')
df_merged_train.drop(('Geo code police', ''), axis=1, inplace=True)

#Test
df_merged_test = pd.merge(abc_test, df_test3, on='Force Name')
df_merged_test.drop(('Geo code police', ''), axis=1, inplace=True)

C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/1921022594.py:2: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  df_merged_train = pd.merge(abc_train, df_train3, on='Force Name')
C:\Users\blaze\Anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/1921022594.py:6: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  df_merged_test = pd.merge(abc_test, df_test3, on='Force Name')


# Add year population and headcount column to merged dataframe

In [373]:
#Train
zz = df_merged_train.copy()
zz = zz.loc[zz.index.repeat(7)]
zz['year']= 2012
zz['population']= 61334
zz['headcount'] = 3181
zz.reset_index(inplace=True)

#Test
yy = df_merged_test.copy()
#yy = yy.loc[yy.index.repeat(1)]
yy['year']= 2019
yy['population']= 61334
yy['headcount'] = 3181
yy.reset_index(inplace=True)

In [374]:
#Train
for i in range (1, zz.shape[0]):
    if zz['index'][i] == zz['index'][i-1]:
        zz['year'][i] = zz['year'][i-1]+1


C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/3389684030.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zz['year'][i] = zz['year'][i-1]+1


In [375]:
for i in range (0, zz.shape[0]):
    if zz['year'][i]==2011:
        zz['population'][i] = zz['population_2011'][i]
        zz['headcount'][i] = zz[('Total (headcount)', 2011)][i]
    if zz['year'][i]==2012:
        zz['population'][i] = zz['population_2012'][i]
        zz['headcount'][i] = zz[('Total (headcount)', 2012)][i]
    if zz['year'][i]==2013:
        zz['population'][i] = zz['population_2013'][i]
        zz['headcount'][i] = zz[('Total (headcount)', 2013)][i]
    if zz['year'][i]==2014:
        zz['population'][i] = zz['population_2014'][i]
        zz['headcount'][i] = zz[('Total (headcount)', 2014)][i]
    if zz['year'][i]==2015:
        zz['population'][i] = zz['population_2015'][i]
        zz['headcount'][i] = zz[('Total (headcount)', 2015)][i]
    if zz['year'][i]==2016:
        zz['population'][i] = zz['population_2016'][i]
        zz['headcount'][i] = zz[('Total (headcount)', 2016)][i]
    if zz['year'][i]==2017:
        zz['population'][i] = zz['population_2017'][i]
        zz['headcount'][i] = zz[('Total (headcount)', 2017)][i]
    if zz['year'][i]==2018:
        zz['population'][i] = zz['population_2018'][i]
        zz['headcount'][i] = zz[('Total (headcount)', 2018)][i]


C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/1698172861.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zz['population'][i] = zz['population_2012'][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/1698172861.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zz['headcount'][i] = zz[('Total (headcount)', 2012)][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/1698172861.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [377]:
#Test
for i in range (0, yy.shape[0]):
    yy['population'][i] = yy['population_2019'][i]
    yy['headcount'][i] = yy[('Total (headcount)', 2019)][i]

C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/1612746822.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yy['population'][i] = yy['population_2019'][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/1612746822.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yy['headcount'][i] = yy[('Total (headcount)', 2019)][i]


In [378]:
zz.drop(['population_2012', 'population_2013','population_2014','population_2015','population_2016','population_2017', 'population_2018', 'population_2019'], axis=1, inplace=True)
yy.drop(['population_2019'], axis=1, inplace=True)
zz.drop(['index',('Total (headcount)', 2012), ('Total (headcount)', 2013), ('Total (headcount)', 2014), ('Total (headcount)', 2015), ('Total (headcount)', 2016),('Total (headcount)', 2017), ('Total (headcount)', 2018)], axis=1, inplace=True)
yy.drop(['index',('Total (headcount)', 2019)], axis=1, inplace=True)

# GDP

In [411]:
uk_gdp = pd.read_excel('GDP_UK_per_head.xlsx')
uk_gdp = uk_gdp.rename(columns={'2019\n[note 3]': '2019'})

In [412]:
ukdpg = uk_gdp.copy()
ukdpg = ukdpg.loc[ukdpg.index.repeat(8)]
ukdpg['year']= 2012
ukdpg['GDP']= 0
ukdpg.reset_index(inplace=True)

In [413]:
for i in range (1, ukdpg.shape[0]):
    if ukdpg['index'][i] == ukdpg['index'][i-1]:
        ukdpg['year'][i] = ukdpg['year'][i-1]+1

C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/1383387713.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukdpg['year'][i] = ukdpg['year'][i-1]+1


In [414]:
for i in range (0, ukdpg.shape[0]):
    if ukdpg['year'][i]==2012:
        ukdpg['GDP'][i] = ukdpg[2012][i]
    if ukdpg['year'][i]==2013:
        ukdpg['GDP'][i] = ukdpg[2013][i]
    if ukdpg['year'][i]==2014:
        ukdpg['GDP'][i] = ukdpg[2014][i]
    if ukdpg['year'][i]==2015:
        ukdpg['GDP'][i] = ukdpg[2015][i]
    if ukdpg['year'][i]==2016:
        ukdpg['GDP'][i] = ukdpg[2016][i]
    if ukdpg['year'][i]==2017:
        ukdpg['GDP'][i] = ukdpg[2017][i]
    if ukdpg['year'][i]==2018:
        ukdpg['GDP'][i] = ukdpg[2018][i]
    if ukdpg['year'][i]==2019:
        ukdpg['GDP'][i] = ukdpg['2019'][i]

C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/1345400967.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukdpg['GDP'][i] = ukdpg[2012][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/1345400967.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukdpg['GDP'][i] = ukdpg[2013][i]
C:\Users\blaze\AppData\Local\Temp/ipykernel_9144/1345400967.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ukdpg['GDP'][i] = ukdpg[2014][i]
C:\U

In [415]:
ukdpg.drop(['index', 'ITL1 Region', 'LA name', 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, '2019'], axis=1, inplace=True)
ukdpg = ukdpg.rename(columns={'LA code': 'district code'})

# Add year column to dataframe from sql

In [416]:
#Train
crime_per_district['year'] = pd.to_datetime(crime_per_district['Month'], format='%Y-%m').dt.year

#Test
crime_per_district2['year'] = pd.to_datetime(crime_per_district2['Month'], format='%Y-%m').dt.year

# Merge everything together

In [417]:
#Train
df_merged_train2 = pd.merge(zz, crime_per_district, on=['year', 'Force Name'])

#Test
df_merged_test2 = pd.merge(yy, crime_per_district2, on=['year', 'Force Name'])

In [418]:
#Train
df_merged_train2.rename(columns={'name_y': 'LSOA name', 'Geo code_y':'LSOA code', 'name_x':'District name', 'Geo code_x':'district code'}, inplace=True)

#Test
df_merged_test2.rename(columns={'name_y': 'LSOA name', 'Geo code_y':'LSOA code', 'name_x':'District name', 'Geo code_x':'district code'}, inplace=True)

In [420]:
#Train
df_merged_train2['month'] = pd.to_datetime(df_merged_train2['Month'], format='%Y-%m').dt.month

#Test
df_merged_test2['month'] = pd.to_datetime(df_merged_test2['Month'], format='%Y-%m').dt.month

In [423]:
import pandas as pd

In [424]:
data = pd.concat([df_merged_test2, df_merged_train2])

MemoryError: Unable to allocate 1.07 GiB for an array with shape (4, 35807513) and data type object

In [228]:
data.reset_index(inplace=True)

In [258]:
data_merged1 = pd.merge(zz2, data, on=['LSOA code', 'year'])

In [269]:
data_merged1

,index_x,LSOA code,year,LSOA population,index_y,District name,district code,Geo code police,Force Name,population,headcount,Month,LSOA name,amount_of_crimes,month
0,1,E01020634,2012,1528.0,29137559,County Durham,E06000047,E23000008,Durham,514261,1383,2012-01,County Durham 001A,3,1
1,1,E01020634,2012,1528.0,29137942,County Durham,E06000047,E23000008,Durham,514261,1383,2012-02,County Durham 001A,4,2
2,1,E01020634,2012,1528.0,29138328,County Durham,E06000047,E23000008,Durham,514261,1383,2012-03,County Durham 001A,11,3
3,1,E01020634,2012,1528.0,29138720,County Durham,E06000047,E23000008,Durham,514261,1383,2012-04,County Durham 001A,4,4
4,1,E01020634,2012,1528.0,29139108,County Durham,E06000047,E23000008,Durham,514261,1383,2012-05,County Durham 001A,4,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112306,2260,E01018744,2019,1615.0,3856876,Warrington,E06000007,E23000006,Cheshire,210014,2037,2019-08,Cheshire West and Chester 037D,10,8
1112307,2260,E01018744,2019,1615.0,3857510,Warrington,E06000007,E23000006,Cheshire,210014,2037,2019-09,Cheshire West and Chester 037D,4,9
1112308,2260,E01018744,2019,1615.0,3858142,Warrington,E06000007,E23000006,Cheshire,210014,2037,2019-10,Cheshire West and Chester 037D,20,10
1112309,2260,E01018744,2019,1615.0,3858773,Warrington,E06000007,E23000006,Cheshire,210014,2037,2019-11,Cheshire West and Chester 037D,8,11


In [270]:
df_data = pd.merge(data_merged1, ukdpg, on=['district code', 'year'])

In [272]:
df_data.drop(['index_x', 'index_y'], axis=1, inplace=True)



,LSOA code,year,LSOA population,District name,district code,Geo code police,Force Name,population,headcount,Month,LSOA name,amount_of_crimes,month,GDP
0,E01020634,2012,1528.0,County Durham,E06000047,E23000008,Durham,514261,1383,2012-01,County Durham 001A,3,1,17751
1,E01020634,2012,1528.0,County Durham,E06000047,E23000008,Durham,514261,1383,2012-02,County Durham 001A,4,2,17751
2,E01020634,2012,1528.0,County Durham,E06000047,E23000008,Durham,514261,1383,2012-03,County Durham 001A,11,3,17751
3,E01020634,2012,1528.0,County Durham,E06000047,E23000008,Durham,514261,1383,2012-04,County Durham 001A,4,4,17751
4,E01020634,2012,1528.0,County Durham,E06000047,E23000008,Durham,514261,1383,2012-05,County Durham 001A,4,5,17751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112306,E01018569,2018,1262.0,Knowsley,E08000011,E23000004,Merseyside,149571,3490,2018-03,Cheshire West and Chester 004B,1,3,27174
1112307,E01018569,2018,1262.0,Liverpool,E08000012,E23000004,Merseyside,494814,3490,2018-03,Cheshire West and Chester 004B,1,3,29215
1112308,E01018569,2018,1262.0,Sefton,E08000014,E23000004,Merseyside,275396,3490,2018-03,Cheshire West and Chester 004B,1,3,18052
1112309,E01018569,2018,1262.0,St. Helens,E08000013,E23000004,Merseyside,180049,3490,2018-03,Cheshire West and Chester 004B,1,3,18420


In [273]:
df_data.to_pickle("df_data.pkl")  

# Train-Test split and encoding

In [338]:
test = df_data[df_data['year'] == 2019]
train = df_data[df_data['year']<2019]

In [278]:
lab = LabelEncoder()

labels_lsoa = df_data['LSOA code'].unique()
lab.fit(labels_lsoa)
train[['LSOA code']] = train[['LSOA code']].apply(lab.fit_transform)
test[['LSOA code']] = test[['LSOA code']].apply(lab.fit_transform)

C:\Users\blaze\Anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [279]:
labels_dist = df_data['district code'].unique()
lab.fit(labels_dist)
train[['district code']] = train[['district code']].apply(lab.fit_transform)
test[['district code']] = test[['district code']].apply(lab.fit_transform)

In [280]:
labels_force = df_data['Force Name'].unique()
lab.fit(labels_force)
train[['Force Name']] = train[['Force Name']].apply(lab.fit_transform)
test[['Force Name']] = test[['Force Name']].apply(lab.fit_transform)

# Random forest implementation

In [327]:
X_train = train[['LSOA code', 'LSOA population', 'headcount', 'month']]
y_train = train['amount_of_crimes']

X_test = test[['LSOA code', 'LSOA population', 'headcount', 'month']]
y_test = test['amount_of_crimes']


In [328]:
cl = RandomForestRegressor(n_estimators = 10, random_state = 42)
cl.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, random_state=42)

In [329]:
pred = cl.predict(X_test)

In [330]:
mean_absolute_error(y_test.values, pred)

11.690628851490143

In [331]:
mean_squared_error(y_test, pred)

655.0219292226747

In [332]:
np.sqrt(mean_squared_error(y_test, pred))


25.593396203370013

In [333]:
r2_score(y_test, pred)

-0.17621019060960408

In [340]:
X_train

,LSOA code,LSOA population,headcount,month
0,1705,1528.0,1383,1
1,1705,1528.0,1383,2
2,1705,1528.0,1383,3
3,1705,1528.0,1383,4
4,1705,1528.0,1383,5
...,...,...,...,...
1112306,1496,1262.0,3490,3
1112307,1496,1262.0,3490,3
1112308,1496,1262.0,3490,3
1112309,1496,1262.0,3490,3


In [334]:
cl.feature_importances_

array([0.75421653, 0.16229236, 0.03600016, 0.04749095])